In [40]:
import geopandas as gpd
from shutil import rmtree
from shapely.geometry import Point
import requests
import zipfile
import os
import osmnx as ox

address = "Washington, D.C., USA"

# Geocode the address to get the coordinates
location = ox.geocode(address)
lat, lon = location[1], location[0]

# Path of the shapefile
shapefile_path = "tiling_schema/WGS84_tile_schema.shp"
extracted_dir = "extracted_files"

# Remove the directory if it already exists
if os.path.exists(extracted_dir):
    rmtree(extracted_dir)

# Load the shapefile
tiles_gdf = gpd.read_file(shapefile_path)

# Ensure the CRS of the tiles is EPSG:4326
if tiles_gdf.crs != "EPSG:4326":
    tiles_gdf = tiles_gdf.to_crs("EPSG:4326")

print(f"Tiles CRS: {tiles_gdf.crs}")

# Create a GeoDataFrame for the point with the same CRS as the tiles
point = Point(lat, lon)
point_gdf = gpd.GeoDataFrame([{'geometry': point}], crs="EPSG:4326")

# Verify in which tile the point is located
current_tile = tiles_gdf[tiles_gdf.contains(point_gdf.geometry.iloc[0])]

# Extract the tile id
if not current_tile.empty:
    tile_id = current_tile.iloc[0]['tile_id']
    print(f"Tile ID: {tile_id}")

    # URL for downloading the tile
    url = f"https://jeodpp.jrc.ec.europa.eu/ftp/jrc-opendata/GHSL/GHS_POP_GLOBE_R2023A/GHS_POP_E2030_GLOBE_R2023A_4326_30ss/V1-0/tiles/GHS_POP_E2030_GLOBE_R2023A_4326_30ss_V1_0_{tile_id}.zip"

    response = requests.get(url)

    # Save the zip file
    if response.status_code == 200:
        zip_path = 'tile_download.zip'
        with open(zip_path, 'wb') as file:
            file.write(response.content)

        # Extract only the .tif file
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            for file in zip_ref.namelist():
                if file.endswith('.tif'):
                    zip_ref.extract(file, extracted_dir)
                    tif_path = os.path.join(extracted_dir, file)
                    new_tif_path = os.path.join(extracted_dir, 'current_tile.tif')
                    os.rename(tif_path, new_tif_path)
        os.remove(zip_path)

Tiles CRS: EPSG:4326
Tile ID: R6_C11
